In [1]:
import torch
import getpass
import os 

from langchain.prompts import PromptTemplate

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

2025-10-07 17:25:58.393574: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 17:25:58.561412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-07 17:25:59.449464: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
from tokeys import keys

In [4]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config = quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True, 
    repetition_penalty = 1.1,
    return_full_text = False, 
)

Device set to use cuda:0


In [12]:
llm = HuggingFacePipeline(pipeline = pipe)

input_ = "Quem foi a primeira pessoa no espaço?"
output = llm.invoke(input_)
print(output)



# Answer:A primeira pessoa no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele voou em uma missão orbital ao redor da Terra como parte do programa espacial soviético Vostok 1. Sua viagem ocorreu de 12 de abril de 1961 e durou cerca de 108 minutos. Ao retornar à Terra, ele pousou com segurança na Rússia, marcando assim o primeiro pouso humano bem-sucedido após um voo espacial tripulado.
